In [1]:
import pandas as pd

df = pd.read_csv('KIC_feature_final.csv')

df.head()

,지역,물건번호,상가구분,월세,보증금,계약면적,전용면적,층수,기타정보,위도,...,C_max,전세금(J),P_min,P_max,예상 매매가,단위 면적당 매매가,center_name,subway_score,bank_score,road_score
0,강원특별자치도 원주시 반곡동,2528873558,지식산업센터,40,500.0,56,36.18,08월 15일,"['4년이내', '중층', '주차가능']",37.327746,...,6.5,9227.272727,7807.692308,9227.272727,8517.482517,235.419638,엔터비즈타워,0.0,100.0,80.65
1,강원특별자치도 원주시 반곡동,2528886552,지식산업센터,40,500.0,76,38.49,07월 15일,"['4년이내', '중층', '주차가능', '엘리베이터']",37.327746,...,6.5,9227.272727,7807.692308,9227.272727,8517.482517,221.290790,엔터비즈타워,0.0,100.0,80.65
2,강원특별자치도 원주시 반곡동,2528480164,지식산업센터,40,500.0,88,44.59,04월 15일,"['4년이내', '지상층(1층제외)', '주차가능']",37.327746,...,6.5,9227.272727,7807.692308,9227.272727,8517.482517,191.017773,엔터비즈타워,0.0,100.0,80.65
3,강원특별자치도 원주시 반곡동,2528476385,지식산업센터,40,500.0,42,42.69,03월 15일,"['4년이내', '지상층(1층제외)', '주차가능', '천장에어컨']",37.327746,...,6.5,9227.272727,7807.692308,9227.272727,8517.482517,199.519384,엔터비즈타워,0.0,100.0,80.65
4,강원특별자치도 원주시 반곡동,2528332848,지식산업센터,90,1000.0,152,76.54,10월 15일,"['4년이내', '중층', '주차가능']",37.327746,...,6.5,20636.363636,17461.538462,20636.363636,19048.951049,248.875765,엔터비즈타워,0.0,100.0,80.65


In [2]:
import pandas as pd
import numpy as np

# 인프라 관련 컬럼들
infra_cols = ['subway_score', 'bank_score', 'road_score']

# 1. 기본 통계량
print("=== 인프라 점수별 기본 통계량 ===")
print(df[infra_cols].describe())

# 2. 변동계수 (Coefficient of Variation) - 상대적 변동성
print("\n=== 변동계수 (CV = std/mean) ===")
cv = df[infra_cols].std() / df[infra_cols].mean()
print(cv)
print(f"평균 변동계수: {cv.mean():.3f}")

# 3. 범위 (Range) - 최대값과 최소값 차이
print("\n=== 범위 (Max - Min) ===")
ranges = df[infra_cols].max() - df[infra_cols].min()
print(ranges)

# 4. 고유값 개수 (Unique Values)
print("\n=== 고유값 개수 ===")
unique_counts = df[infra_cols].nunique()
print(unique_counts)
print(f"전체 데이터 수: {len(df)}")
print(f"고유값 비율: {unique_counts / len(df)}")

# 5. 분위수 범위 (IQR - Interquartile Range)
print("\n=== 분위수 범위 (IQR = Q3 - Q1) ===")
iqr = df[infra_cols].quantile(0.75) - df[infra_cols].quantile(0.25)
print(iqr)

# 6. 엔트로피 (정보 다양성) - 연속형 데이터를 구간화해서 계산
from scipy.stats import entropy
print("\n=== 엔트로피 (정보 다양성) ===")
for col in infra_cols:
    # 10개 구간으로 나누어 히스토그램 생성
    hist, _ = np.histogram(df[col].dropna(), bins=10)
    # 확률 분포로 변환
    prob = hist / hist.sum()
    # 엔트로피 계산 (0에 가까우면 다양성 낮음, 높으면 다양성 높음)
    ent = entropy(prob[prob > 0])  # 0 제거 후 계산
    print(f"{col}: {ent:.3f}")

# 7. 종합 다양성 지수 (custom metric)
print("\n=== 종합 다양성 지수 ===")
def diversity_index(series):
    """다양성 지수: CV + 정규화된 고유값 비율 + 정규화된 엔트로피"""
    cv = series.std() / series.mean() if series.mean() != 0 else 0
    unique_ratio = series.nunique() / len(series)
    

=== 인프라 점수별 기본 통계량 ===
       subway_score    bank_score    road_score
count  27397.000000  27397.000000  27397.000000
mean      39.589279     78.767748     72.863671
std       26.999731     40.895938     15.451923
min        0.000000      0.000000      0.000000
25%        4.200000    100.000000     59.580000
50%       51.770000    100.000000     76.610000
75%       54.520000    100.000000     84.380000
max       95.620000    100.000000    100.000000

=== 변동계수 (CV = std/mean) ===
subway_score    0.681996
bank_score      0.519196
road_score      0.212066
dtype: float64
평균 변동계수: 0.471

=== 범위 (Max - Min) ===
subway_score     95.62
bank_score      100.00
road_score      100.00
dtype: float64

=== 고유값 개수 ===
subway_score    210
bank_score        2
road_score      209
dtype: int64
전체 데이터 수: 27397
고유값 비율: subway_score    0.007665
bank_score      0.000073
road_score      0.007629
dtype: float64

=== 분위수 범위 (IQR = Q3 - Q1) ===
subway_score    50.32
bank_score       0.00
road_score      24.80
d

In [1]:
import pandas as pd

# 엑셀 파일 읽기 및 행 수 확인
df_excel = pd.read_excel('파이널원본_업데이트.xlsx')

print(f"파이널원본_업데이트.xlsx 행 수: {len(df_excel)}")
print(f"컬럼 수: {df_excel.shape[1]}")
print(f"전체 형태: {df_excel.shape}")

# 추가 정보
print(f"\n컬럼명: {list(df_excel.columns)}")
print(f"첫 5행 미리보기:")
print(df_excel.head())

파이널원본_업데이트.xlsx 행 수: 1101
컬럼 수: 13
전체 형태: (1101, 13)

컬럼명: ['회사명', '대표업종코드', '총자산', '자본금', '순이익률', '자기자본비율', '영업이익', '1인당매출', '종업원합계', '지역', '전용면적', '인프라', '단위 면적당 매매가']
첫 5행 미리보기:
         회사명  대표업종코드         총자산            자본금       순이익률     자기자본비율  \
0   (주)가을디에스      26  15850399.0  500000.000000   0.907248  21.151105   
1  (주)감속기케이원      29    918063.0  300000.000000 -11.149515 -93.056894   
2      (주)감수      26   1379618.0  150000.000000   9.555012  55.516872   
3      (주)거목      26    310772.0   33833.333333   7.987850  33.039327   
4      (주)거봉      26  34440653.0  200000.000000   1.565525  26.992449   

           영업이익         1인당매출  종업원합계             지역      전용면적     인프라  \
0  3.835770e+05  2.624373e+06    6.0      서울특별시영등포구   152.630  78.877   
1  5.977267e+04  1.483327e+05    1.0      경기도김포시양촌읍   179.010  42.036   
2  3.934303e+05  6.238946e+05    6.0  경상남도창원시의창구팔용동   371.513  46.876   
3  5.518633e+04  1.718071e+05    5.0      경기도김포시구래동    57.120  75.347   
4  1.029670e+06